# Lab 9: Dissecting racial bias in a medical risk score

---

ML Failures lab: Dissecting Racial Bias in a Medical Risk Score by Nick Merrill, Inderpal Kaur, Samuel Greenberg is licensed under [CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0). Edits by Emily Aiken.

**Students** can [provide feedback here](https://docs.google.com/forms/d/1jI8oXRkqD1l1ARuZR1y9W_qkOystPr-YEyywNDez46M/edit?ts=5efa772a&dods).

# Background

To effectively manage patients, health systems often need to estimate particular
patients' health risks. Using quantitative measures, or "risk scores,"
healthcare providers can prioritize patients and allocate resources to patients
who need them most.

In this lab, we examine an algorithm widely-used in industry to establish quantitative risk scores for patients. This algorithm uses *medical cost* (i.e., the amount a patient spends on medical care) as a proxy for risk. Through analysis of this data, we will discover how this algorithm embeds a bias against Black patients, undervaluing their medical risk relative to White patients. Crucially, this bias is not immediately visible when comparing medical costs across White and Black pateints.

The key insight of [Obermeyer et al's work](https://science.sciencemag.org/content/366/6464/447) is that bias frequently slips into algorithmic systems unnoticed, particularly when sensitive characteristics (such as race) are ommitted or backgrounded in the data science process. In this case, bias in algorithms affects people's lives very concretely: the bias in the algorithm described here would make it more difficult for Black patients to recieve the care they need.

In this lab, we will learn how to uncover bias in algorithms, using the medical risk score example from Obermeyer et al's paper.

## Agenda

- Investigate relationship between medical cost and risk by race
- Investigate relationship between chronic illness and risk by race
- Investigate interactions between medical cost and chronic illness
- Discussion

In [1]:
# import statements
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker
from statsmodels.nonparametric.smoothers_lowess import lowess
%matplotlib inline

## Medical cost and risk

To help hospitals and insurance companies identify patients who should qualify for “high-risk care management” programs, the algorithm assigns each patient a risk score. It predicts a patient’s total medical expenditure based on data from insurance claims (age, sex, diagnosis codes, etc.) and uses this variable as a proxy for health care needs.  Patient risk scores are then generated as functions of their predicted expenditures.

If the model is calibrated across race in terms of risk score and expenditure, Black and White patients with a given risk score should have similar total medical expenditures, on average.

To see if this is true, we will generate a graph that shows the mean total medical expenditure by race given a risk score percentile.

First, let's load our medical data. This data represents the estimated risk across patients, along with other details about each patient. You can [grab the data here](https://github.com/daylight-lab/mlfailures/blob/master/health-care-bias-lab.csv). (If you're using this lab as a Jupyter notebook on your computer, place the file in the same directory as your lab. If you're using Google Colab, Click the Files menu in the sidebar and press the upload file button to upload this CSV.)

In [2]:
# Load data
data = pd.read_csv('health-care-bias-lab.csv')
data.head()

,risk_score_t,program_enrolled_t,cost_t,cost_avoidable_t,bps_mean_t,ghba1c_mean_t,hct_mean_t,cre_mean_t,ldl_mean_t,race,...,trig_min-high_tm1,trig_min-normal_tm1,trig_mean-low_tm1,trig_mean-high_tm1,trig_mean-normal_tm1,trig_max-low_tm1,trig_max-high_tm1,trig_max-normal_tm1,gagne_sum_tm1,gagne_sum_t
0,1.987430,0,1200.0,0.0,NaN,5.4,NaN,1.110000,194.0,white,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7.677934,0,2600.0,0.0,119.0,5.5,40.4,0.860000,93.0,white,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,4.0,3.0
2,0.407678,0,500.0,0.0,NaN,NaN,NaN,NaN,NaN,white,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.798369,0,1300.0,0.0,117.0,NaN,NaN,NaN,NaN,white,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,17.513165,0,1100.0,0.0,116.0,NaN,34.1,1.303333,53.0,white,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [3]:
# TODO: Add a coolumn to the dataframe called "risk percentile" with the percentile of the risk score 
# (`risk_score_t`) for each observation
# HINT: Try using pandas qcut

In [4]:
# TODO create dataframe called `group_cost` with the mean total medical expenditure (`cost_t`) for each race (`race`) 
# at each risk percentile (`risk_percentile`)

In [5]:
# TODO: Divide `group_cost` into two dataframes based on race. Call the two dataframes `b_cost` and `w_cost`.

In [6]:
# TODO: Plot risk percentile against cost, splitting on race

**QUESTION: What do you notice about the relationship between medical expenditure and risk score by race? Can you conclude from this data that the model is fair or not fair?**

## Chronic illness and risk

Next, we will check to see if the model is calibrated across groups in terms of risk score and chronic illness. In other words: for a given risk score, do Black and white patients have the same "level" of health? 

If cost is a good proxy for need, we would expect this graph to be as balanced as the prior graph.  In other words, health care cost should not vary conditional on health between groups.

In [7]:
# TODO Group the data by `risk_percentile` and `race`. Take the mean number of chronic illnesses (`gagne_sum_t`) in 
# each group of race + risk percentile. Call that dataframe `grouped_by_race`.

In [8]:
# TODO Plot risk percentile against average number of chronic ilnesses, splitting on race

**QUESTION: What do you notice about the relationship between chronic illness and risk score by race?  Can you conclude from this data that the model is fair or not fair?**

**QUESTION: How might we test for a statistically significant difference in chronic illnesses by race, conditioned on risk percentile?**

In [9]:
# TODO: Fit two linear regressions for chronic illness regressed on risk percentile (one for each race). 
# What do you notice about the coefficients? 

## Interactions between cost and illness

Our work above shows us that a Black patient and a White patient with the same risk score tend to spend the same amount on medical care on average, yet the Black patient tends to have more chronic illnesses.

To understand this interaction, generate a graph that shows the mean total medical expenditure by race, given the number of chronic illnesses.

In [10]:
# TODO: Add a column of illness percentiles to the dataframe called 'illness_percentile'

In [11]:
# TODO: Group the data by `illness_percentile` and `race`. Take the mean total medical expenditure (`cost_t`) for 
# each group of race + illness percentile. Call the dataframe called `illnesses`.

In [12]:
# TODO divide illnesses into two dataframes based on race: `illness_b` and `illness_w`

In [13]:
# TODO create a scatterplot of illness percentile against costs, splitting on race

**QUESTION: What can you conclude about the relationship between cost and chronic illness? Why might this relationship exist? What are consequences for the risk score model?**

**OPTIONAL**: Fit a LOWESS (locally weighted scatterplot smoothing) model to the scatterplot. Or perhaps there's some other model you'd like to try out.

## Conclusions and takeaways

Even systems that appear balanced across racial groups at first glance may belie underlying biases in the datasets. Thus, seemingly unbiased predictors can in fact be highly correlated with a biasing variable such as race, gender, income or other relational characteristics.

In this example, bias emerged from using an indicator of need (cost) that was itself influenced by race. Biased estimation of need between races resulted.

To better understand the ways in which race influences health care cost, here is a segment from Obermeyer et al’s paper:
 
>The literature broadly suggests two main potential channels. **First, poor patients face substantial barriers to accessing health care, even when enrolled in insurance plans.** Although the population we study is entirely insured, there are many other mechanisms by which poverty can lead to disparities in use of health care: geography and differential access to transportation, competing demands from jobs or child care, or knowledge of reasons to seek care (1-3). To the extent that race and socioeconomic status are correlated, these factors will differentially affect Black patients. **Second, race could affect costs directly via several channels: direct (“taste-based”) discrimination, changes to the doctor–patient relationship, or others. A recent trial randomly assigned Black patients to a Black or White primary care provider and found significantly higher uptake of recommended preventive care when the provider was Black (4).** This is perhaps the most rigorous demonstration of this effect, and it fits with a larger literature on potential mechanisms by which race can affect health care directly. For example, it has long been documented that Black patients have reduced trust in the health care system (5), a fact that some studies trace to the revelations of the Tuskegee study and other adverse experiences (6). A substantial literature in psychology has documented physicians’ differential perceptions of Black patients, in terms of intelligence, affiliation (7), or pain tolerance (8). **Thus, whether it is communication, trust, or bias, something about the interactions of Black patients with the health care system itself leads to reduced use of health care. The collective effect of these many channels is to lower health spending substantially for Black patients, conditional on need—a finding that has been appreciated for at least two decades (9).**

## Reflection Questions

Here are two final open-ended questions for you to answer.

1. How could we use the data we have to create new proxies for health needs that may be less biased than medical costs?


2. What are other applications of prediction algorithms where this type of bias may also arise?


## References

1. K. Fiscella, P. Franks, M. R. Gold, C. M. Clancy, JAMA 283, 2579–2584 (2000).
2. N. E. Adler, K. Newman, Health Aff. 21, 60–76 (2002).
3. N. E. Adler, W. T. Boyce, M. A. Chesney, S. Folkman, S. L. Syme, JAMA 269, 3140–3145 (1993).
4. M. Alsan, O. Garrick, G. C. Graziani, “Does diversity matter for health? Experimental evidence from Oakland” (National Bureau of Economic Research, 2018).
5. K. Armstrong, K. L. Ravenell, S. McMurphy, M. Putt, Am. J. Public Health 97, 1283–1289 (2007).
6. M. Alsan, M. Wanamaker, Q. J. Econ. 133, 407–455 (2018).
7. M. van Ryn, J. Burke, Soc. Sci. Med. 50, 813–828 (2000).
8. K. M. Hoffman, S. Trawalter, J. R. Axt, M. N. Oliver, Proc. Natl. Acad. Sci. U.S.A. 113, 4296–4301 (2016).
9. J. J. Escarce, F. W. Puffer, in Racial and Ethnic Differences in the Health of Older Americans (National Academies Press, 1997), chap. 6; www.ncbi.nlm.nih.gov/books/ NBK109841/.

# Feedback

**Instructors**: Please [provide feedback](https://docs.google.com/forms/d/1UuUVBBMTU_2aMvzsGnTR_4i1w3F6tLaaqdIr7dQrgSI/edit?ts=5efa771b&dods) to help improve this lab.

**Students**: Please [provide feedback](https://docs.google.com/forms/d/1jI8oXRkqD1l1ARuZR1y9W_qkOystPr-YEyywNDez46M/edit?ts=5efa772a&dods) to help improve this lab.
